
## Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime, re

In [12]:
import time
time.sleep(0.2)

In [13]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
KEYWORDS = ['python', 'парсинг']

In [14]:
posts = soup.find_all('article', class_='post')


In [15]:
habr_python = pd.DataFrame()
for word in KEYWORDS:
    for post in posts:
        post_lower = post.text.lower()
        if word in post_lower:
            date = post.find('span', class_="post__time").text
            title = post.find('a', class_='post__title_link')
            site = title.attrs.get('href')
            #date = title.attrs.get('href')
            row = {'date': date, 'title': title.text, 'site': site}
            habr_python = pd.concat([habr_python, pd.DataFrame([row])])  
habr_python.reset_index()

,index,date,title,site
0,0,сегодня в 15:41,Машинное обучение: ансамбль смешивания на Python,https://habr.com/ru/company/skillfactory/blog/...



### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [16]:
titles = list(map(lambda x: x.find('a', class_='post__title_link'), posts))
links = list(map(lambda x: x.attrs.get('href'), titles))
links

['https://habr.com/ru/post/531462/',
 'https://habr.com/ru/company/huawei/blog/531460/',
 'https://habr.com/ru/post/531458/',
 'https://habr.com/ru/company/rshb/blog/531456/',
 'https://habr.com/ru/company/ruvds/blog/531198/',
 'https://habr.com/ru/post/531452/',
 'https://habr.com/ru/company/skillfactory/blog/531416/',
 'https://habr.com/ru/post/531450/',
 'https://habr.com/ru/post/531446/',
 'https://habr.com/ru/post/531442/',
 'https://habr.com/ru/post/531438/',
 'https://habr.com/ru/company/piter/blog/531434/',
 'https://habr.com/ru/post/531432/',
 'https://habr.com/ru/post/531400/',
 'https://habr.com/ru/company/ruvds/blog/530944/',
 'https://habr.com/ru/post/531426/',
 'https://habr.com/ru/company/audiomania/blog/526010/',
 'https://habr.com/ru/post/531410/',
 'https://habr.com/ru/post/526804/',
 'https://habr.com/ru/post/531384/']

In [17]:
for link in links:
    soup2 = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    posts2 = soup.find_all('article', class_='post')
   

In [20]:
habr_posts_python = pd.DataFrame()
    
for word in KEYWORDS:
    for post in posts2:
        post_lower = post.text.lower()
        if word in post_lower:
            date = post.find('span', class_="post__time").text
            title = post.find('a', class_='post__title_link')
            site = title.attrs.get('href')
            contents = post.text.lower()
            #print(contents)
            row2 = {'date': date, 'title': title.text, 'site': site}
            habr_posts_python = pd.concat([habr_posts_python, pd.DataFrame([row2])])  
            
habr_posts_python.reset_index()

,index,date,title,site
0,0,сегодня в 15:41,Машинное обучение: ансамбль смешивания на Python,https://habr.com/ru/company/skillfactory/blog/...



## Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.


In [21]:
EMAIL = ['vogue13@mail.ru', 'ukrashaem@gmail.com']

data = {'emailAddresses': EMAIL}
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Version': '0',  'Vaar-Header-App-Product': 'hackcheck-web-avast'}

res = requests.post(URL, json=data, headers=headers)

info = res.json().copy()

In [22]:
breach_db = pd.DataFrame()

for breach in info['breaches']:
    breach_date = datetime.datetime.strptime(res.json()['breaches'][breach]['publishDate'],'%Y-%m-%dT%H:%M:%SZ')
    breach_date =  breach_date.strftime('%d.%m.%Y')
    breach_source = res.json()['breaches'][breach]['site']
    description = res.json()['breaches'][breach]['description']
    

    for mail in info['summary']:
        if int(breach) in info['summary'][mail]['breaches']:
            breach_mail = mail
            
    db_row = {'email': breach_mail, 'date': breach_date, 'source': breach_source, 'description': description}
    breach_db = pd.concat([breach_db, pd.DataFrame([db_row])]) 

breach_db.reset_index()


,index,email,date,source,description
0,0,ukrashaem@gmail.com,26.10.2016,twitter.com,Login credentials for over 32 Million Twitter ...
1,0,vogue13@mail.ru,01.11.2016,qip.ru,"In 2011, Russian instant messaging service pro..."
2,0,ukrashaem@gmail.com,13.06.2019,canva.com,"In May 2019, graphic-design site Canva's datab..."
3,0,vogue13@mail.ru,25.10.2016,last.fm,"In March 2012, the British music streaming ser..."
4,0,ukrashaem@gmail.com,23.05.2019,livejournal.com,"In 2017, social network LiveJournal's database..."



## Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [10]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
TOKEN = 'X'
VERSION = '5.103'
SLEEP = 0.33

In [11]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain' : 'netology',
    'count': 50
}

In [12]:
res = requests.get(NEWSFEED_REQUEST, params)
res

<Response [200]>

In [56]:
res.json()

{'response': {'count': 7366,
  'items': [{'id': 64430,
    'from_id': -30159897,
    'owner_id': -30159897,
    'date': 1606723210,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'В воздухе уже витает дух Нового года, а мы устраиваем предновогодний digital-марафон 🎉 \n \nКаждую неделю мы будем разыгрывать призы от Нетологии и наших партнёров, а накануне Нового года — 28 декабря, подведём окончательные итоги. Самые активные участники сообщества по итогам 4-х недель получат гранты на обучение в Нетологии на сумму 30 000, 15 000 и 10 000 рублей. \n \n🥳 На этой неделе мы разыграем 5 подписок на месяц фитнеса от наших партнёров — онлайн фитнес-клуба [club33049742|FitStars]. Можно немного отдохнуть от зарядки для мозга на наших курсах и размяться 😉 \n\nНемного о правилах и призах: \n🔹Каждую неделю несколько самых активных участников получат приз. Нельзя дважды оказаться в числе призёров: если вы заняли 1-е место в рейтинге повторно, приз переходит участнику, который находится с

In [25]:
vk_netology = pd.DataFrame()

for i in range(50):
    date = datetime.datetime.fromtimestamp(res.json()['response']['items'][i]['date']).strftime("%d/%m/%Y") 
    text = re.sub(r'[\s+]', ' ',  res.json()['response']['items'][i]['text'])
    row = {'date': date, 'text': text}
    vk_netology = pd.concat([vk_netology, pd.DataFrame([row])])
    
vk_netology

,date,text
0,30/11/2020,"В воздухе уже витает дух Нового года, а мы уст..."
0,02/12/2020,🔥 19 декабря пройдёт бесплатная онлайн-конфере...
0,01/12/2020,"С тире или без? А может, дефис поставить? Раз..."
0,01/12/2020,"Что надо знать и уметь, чтобы работать с данны..."
0,30/11/2020,📆 Составляем адвент-календарь взрослого челове...
0,30/11/2020,🔥 Запустили бесплатный курс «Как перенести сво...
0,29/11/2020,"Неопределённость, непредсказуемые ситуации, ре..."
0,28/11/2020,Онлайн-мероприятия декабря: конференции для ди...
0,28/11/2020,"Онлайн-курсы становятся всё популярнее, рынок ..."
0,28/11/2020,"Собрали статьи, которые помогут сделать работу..."
